# Read Before

- https://www.kaggle.com/code/raddar/icr-competition-analysis-and-findings/notebook
- https://www.tensorflow.org/guide/core/logistic_regression_core

Plan:
- [x] Feature Engineering (1 day)
- [x] CV and Model Selection (1 day)
- [x] Validation (1 day)
- [x] Review
- [ ] Found that some variables are constant as they represent some features for categroical column which takes only two values, so makes sense to drop them in order not overtrain. I will make split -> make two versions of model which is run if Group A (on all data), which is Group B ( which finetuned after )
- [ ] Make Artefacts


## Install Private utils

In [1]:
!pip install auto-ml --no-index --find-links=file:///kaggle/input/private-utils

Looking in links: file:///kaggle/input/private-utils
Processing /kaggle/input/private-utils/auto_ml-0.1.0-py3-none-any.whl


In [2]:
from auto_ml.encoding_and_transforms import WOENumericalComplex

# Import the libraries

In [3]:
import tensorflow as tf
import tensorflow_decision_forests as tfdf

from keras import metrics
from keras import backend as K

import keras_tuner as kt

import pandas as pd
from pandas.api.types import is_numeric_dtype

import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.metrics import log_loss
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import Normalizer
from sklearn.model_selection import StratifiedKFold, KFold

import warnings
from tqdm.notebook import tqdm

pd.set_option('display.max_rows', 500)

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

In [4]:
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)

# Load the Dataset

In [5]:
dataset_df = pd.read_csv('/kaggle/input/icr-identify-age-related-conditions/train.csv', index_col='Id')
dataset_df.columns = dataset_df.columns.str.rstrip()
print("Full train dataset shape is {}".format(dataset_df.shape))

dataset_test_df = pd.read_csv('/kaggle/input/icr-identify-age-related-conditions/test.csv', index_col='Id')
dataset_test_df.columns = dataset_test_df.columns.str.rstrip()
print("Full test dataset shape is {}".format(dataset_test_df.shape))

Full train dataset shape is (617, 57)
Full test dataset shape is (5, 56)


The data is composed of 58 columns and 617 entries. We can see all 58 dimensions(results will be truncated since the number of columns is big) of our dataset by printing out the first 5 entries using the following code:

# Feature Engineering

## Compute Basic Info

In [6]:
def compute_basic_stats(columns, df):
    out = {}
    
    for i in tqdm(columns):
        mask = df[i].notna()
        
        out[i] = {'nunique':df[i].nunique(),
                  'na_share':round(100*df[i].isna().sum()/df[i].count(),1),
                  'dtype':df[i].dtype
                 }
        if is_numeric_dtype(df[i]):
            out[i]['correlation'] = round(np.corrcoef(x=df.loc[mask,i],y=df.loc[mask,'Class'])[0,1],2)
            out[i]['min'] = df.loc[mask,i].min()
            out[i]['max'] = df.loc[mask,i].max()
            out[i]['std'] = df.loc[mask,i].std()
            out[i]['mean'] = df.loc[mask,i].mean()
            i_lorreg = LogisticRegression()
            X = df.loc[mask,i].values.reshape(-1,1)
            y = df.loc[mask,'Class'].values
            i_lorreg.fit(X=X, y=y)
            y_pred = i_lorreg.predict(X)
            out[i]['logloss'] = log_loss(y_true=y, y_pred=y_pred)
            
            
    out = pd.DataFrame(out).T
    
    out = out.sort_values('logloss',ascending=True)
    
    return out

basic_stats_1 = compute_basic_stats(
    columns=[i for i in dataset_df.columns if i not in ["Id","Class"]],         
    df=dataset_df)

  0%|          | 0/56 [00:00<?, ?it/s]

* Only one variable looks constrant over the target -> better to omit it.
* Realised Better to add LogLoss metric for each feature -> loggloss


## Create Features

In [7]:
def preprocess(train: pd.DataFrame, test: pd.DataFrame, stats: pd.DataFrame) -> (pd.DataFrame, pd.DataFrame, dict):
    info = dict()
    numeric_features = stats[stats.logloss.notna()].index.tolist() # Cause for EJ logloss is null
    train_out = train.copy()
    test_out = test.copy()
    
    out_features = list()
    
    # Make WoE Columns
    for i in tqdm(numeric_features, 'WoE Encoding: '):
        tmp_woe = WOENumericalComplex()
        tmp_woe.fit(x=train_out[i], y=train_out['Class'])
        train_out[i + '_WoE'] = tmp_woe.transform(X=train_out[i])
        test_out[i + '_WoE'] = tmp_woe.transform(X=test_out[i])
        out_features.append(i + '_WoE')
    
    
    # Make NA columns
    for i in tqdm(['DU', 'FC', 'FS', 'CC', 'FL', 'GL', 'CB', 'EL', 'BQ'], 'Split by NA: '):
        train_out[i+'_na'] = np.where(train_out[i].isna(),1,0)
        test_out[i+'_na'] = np.where(test_out[i].isna(),1,0)
        out_features.append(i + '_na')
    
    # Basic Logic -> normalise
    for i in tqdm(numeric_features,'Normalise Numeric: '):
        if stats.loc[i,'correlation'] > 0:
            na_value = stats.loc[i,'max']
        else:
            na_value = stats.loc[i,'min']

        train_out[i] = train_out[i].fillna(na_value)
        test_out[i] = test_out[i].fillna(na_value)

        train_out[i] = (train_out[i]-stats.loc[i,'mean'])/stats.loc[i,'std']
        test_out[i] = (test_out[i]-stats.loc[i,'mean'])/stats.loc[i,'std']

        out_features.append(i)
            
            
    # Addition EJ -> has only two values, so if EJ == 'A'
    train_out['EJ' + '_A'] = np.where(train_out['EJ'] == 'A',1,0)
    test_out['EJ' + '_A'] = np.where(test_out['EJ'] == 'A',1,0)
    out_features.append('EJ' + '_A')
    
    return train_out,test_out,out_features


train_out,test_out,features = preprocess(train=dataset_df, test=dataset_test_df, stats=basic_stats_1)

WoE Encoding:   0%|          | 0/55 [00:00<?, ?it/s]

Split by NA:   0%|          | 0/9 [00:00<?, ?it/s]

Normalise Numeric:   0%|          | 0/55 [00:00<?, ?it/s]

In [8]:
basic_stats_2 = compute_basic_stats(
    columns=[i for i in train_out.columns if i not in ["Id","Class"]],         
    df=train_out)

basic_stats_2

  0%|          | 0/121 [00:00<?, ?it/s]

,nunique,na_share,dtype,correlation,min,max,std,mean,logloss
DU_WoE,4,0.0,float64,-0.52,-2.678782,2.977892,1.042059,0.286027,4.790242
FL_WoE,4,0.0,float64,-0.42,-2.045013,2.977892,0.86161,0.214552,5.374418
DI_WoE,3,0.0,float64,-0.37,-2.361247,0.382521,0.698812,0.124925,5.432836
DA_WoE,3,0.0,float64,-0.37,-2.093932,0.463416,0.719046,0.14452,5.549671
GL_WoE,4,0.0,float64,-0.4,-1.805209,2.977892,0.865805,0.230269,5.783341
FD_WoE,3,0.0,float64,-0.33,-1.996604,0.379845,0.635814,0.115675,5.783341
EH_WoE,3,0.0,float64,-0.32,-2.061142,0.343225,0.622624,0.111704,5.841759
EH,127,0.0,float64,0.18,-0.1635,22.876681,1.0,-0.0,5.900177
AM,605,0.0,float64,0.24,-0.513293,8.483647,1.0,0.0,5.900177
BC_WoE,3,0.0,float64,-0.3,-1.9811,0.486565,0.630381,0.116679,5.900177


# Train Model

Today, we will use the defaults to create the Random Forest Model. By default the model is set to train for a classification task.
We will train a model for each fold and after training we will store the model and metrics. Here, we have chosen `accuracy` and `binary_crossentropy` as the metrics.

In [9]:
def train_model(train: pd.DataFrame, test: pd.DataFrame, features: list, label = "Class",
                n_splits: int = 6,
                model_obj = tfdf.keras.RandomForestModel,
                model_kwargs = dict(),
                model_compile_kwargs = dict()) -> (pd.DataFrame, pd.DataFrame, pd.DataFrame, dict):

    # Create a dataframe of required size with zero values.
    oof = pd.DataFrame(data=np.zeros((len(train.index),1)), index=train.index)
    submition = pd.DataFrame(data=np.zeros((len(test.index),n_splits)),index=test.index)

    # Create an empty dictionary to store the models trained for each fold.
    models = {}

    # Create empty dict to save metircs for the models trained for each fold.
    accuracy = {}
    cross_entropy = {}
    balanced_logloss = {}
    
    # Calculate the number of samples for each label.
    neg, pos = np.bincount(train[label])
    total = neg + pos
    weight_for_0 = (1 / neg) * (total / 2.0)
    weight_for_1 = (1 / pos) * (total / 2.0)
    class_weight = {0: weight_for_0, 1: weight_for_1}

    print('Weight for class 0: {:.2f}'.format(weight_for_0))
    print('Weight for class 1: {:.2f}'.format(weight_for_1))
    
    # Names of columns
    submition_ds = tfdf.keras.pd_dataframe_to_tf_dataset(test)
    # Loop through each fold
    skf = StratifiedKFold(n_splits=n_splits)
    
    for i, (train_index, valid_index) in enumerate(skf.split(X=train,y=train['Class'])):
            print('##### Fold',i+1)

            # Fetch values corresponding to the index 
            train_df = train.iloc[train_index]
            valid_df = train.iloc[valid_index]
            valid_ids = valid_df.index.values

            # Select only feature columns for training.
            train_df = train_df[features+[label]]
            valid_df = valid_df[features+[label]]

            # We need to convert the datatset from Pandas format (pd.DataFrame)
            # into TensorFlow Datasets format (tf.data.Dataset).
            train_ds = tfdf.keras.pd_dataframe_to_tf_dataset(train_df, label=label)
            valid_ds = tfdf.keras.pd_dataframe_to_tf_dataset(valid_df, label=label)

            # Define & Train the model and metrics
            model = model_obj(**model_kwargs)
            model.compile(**model_compile_kwargs) 
            model.fit(x=train_ds, class_weight=class_weight)

            # Store the model
            models[f"fold_{i+1}"] = model

            # Predict value for validation/Submition data
            # Store the predictions in oof dataframe
            oof.loc[valid_ids, 0] = model.predict(x=valid_ds).flatten()
            submition[i] = model.predict(x=submition_ds).flatten() 

            # Evaluate and store the metrics in respective dicts
            evaluation = model.evaluate(x=valid_ds,return_dict=True)
            accuracy[f"fold_{i+1}"] = evaluation["accuracy"]
            cross_entropy[f"fold_{i+1}"] = evaluation["binary_crossentropy"]
            balanced_logloss[f"fold_{i+1}"] = evaluation["balanced_logloss"]
            
    return submition,models,accuracy,cross_entropy,balanced_logloss

def balanced_logloss(y_true: tf.Tensor, y_pred: tf.Tensor) -> tf.Tensor:
    # y_true is prob that y is equals to 1, we assume that final probs would be P(class_1) = 1 - P(class_0)
    y_pred_1 = y_pred
    y_pred_0 = 1-y_pred

    log_y_pred_1 = tf.reshape(K.log(y_pred_1),[-1,1])
    log_y_pred_0 = tf.reshape(K.log(y_pred_0),[-1,1])

    y_1 = tf.reshape(y_true,[1,-1])
    y_0 = (y_1-1)*(-1)

    logloss_1 = -K.dot(y_1,log_y_pred_1)[0][0]/K.sum(y_1)
    logloss_0 = -K.dot(y_0,log_y_pred_0)[0][0]/K.sum(y_0)

    av_logloss = (logloss_1+logloss_0)/2
    
    return av_logloss

def print_average_accuracy(models,cross_entropy,accuracy,balanced_logloss):
    average_loss = 0
    average_acc = 0
    average_bal = 0

    for _model in  models:
        average_loss += cross_entropy[_model]
        average_acc += accuracy[_model]
        average_bal += balanced_logloss[_model]
        
        print(f"{_model}: acc: {accuracy[_model]:.4f} loss: {cross_entropy[_model]:.4f} balanced loss: {balanced_logloss[_model]:.4f}")

    print(f"\nAverage accuracy: {average_acc/len(models):.4f}  Average loss: {average_loss/len(models):.4f} Average balanced loss: {average_bal/len(models):.4f}")

In [10]:
# RandomForestModel
submition_1, model_1,accuracy_1,cross_entropy_1, bal_logloss_1 = train_model(
    train=train_out,test=test_out, features=features,                                             
    n_splits=10,                    
    model_obj=tfdf.keras.RandomForestModel,           
    model_kwargs=dict(max_depth=6, num_trees=1000),                                                  
    model_compile_kwargs=dict(metrics=[metrics.accuracy, metrics.binary_crossentropy,balanced_logloss]))
# metrics
print('Type 1')
print_average_accuracy(models=model_1, cross_entropy=cross_entropy_1, accuracy=accuracy_1, balanced_logloss=bal_logloss_1)

Weight for class 0: 0.61
Weight for class 1: 2.86
##### Fold 1
Use /tmp/tmp1d37tpbc as temporary training directory
Reading training dataset...
Training dataset read in 0:00:09.244720. Found 555 examples.
Training model...
Model trained in 0:00:00.777716
Compiling model...


[INFO 23-07-17 21:15:50.2552 UTC kernel.cc:1242] Loading model from path /tmp/tmp1d37tpbc/model/ with prefix a3f9f43dadc54420
[INFO 23-07-17 21:15:50.3916 UTC decision_forest.cc:660] Model loaded with 1000 root(s), 34210 node(s), and 113 input feature(s).
[INFO 23-07-17 21:15:50.3916 UTC abstract_model.cc:1311] Engine "RandomForestOptPred" built
[INFO 23-07-17 21:15:50.3918 UTC kernel.cc:1074] Use fast generic engine


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: could not get source code
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Model compiled.
1/1 [==============================] - 1s 1s/step - loss: 0.0000e+00 - accuracy: 0.0000e+00 - binary_crossentropy: 0.2010 - balanced_logloss: 0.2942
##### Fold 2
Use /tmp/tmpusuck7ii as temporary training directory
Reading training dataset...
Training dataset read in 0:00:01.993676. Found 555 examples.
Training model...
Model trained in 0:00:00.744068
Compiling model...


[INFO 23-07-17 21:15:59.2011 UTC kernel.cc:1242] Loading model from path /tmp/tmpusuck7ii/model/ with prefix 53e8e248eb244f7b
[INFO 23-07-17 21:15:59.3376 UTC decision_forest.cc:660] Model loaded with 1000 root(s), 34512 node(s), and 113 input feature(s).
[INFO 23-07-17 21:15:59.3380 UTC kernel.cc:1074] Use fast generic engine


Model compiled.
1/1 [==============================] - 1s 964ms/step - loss: 0.0000e+00 - accuracy: 0.0000e+00 - binary_crossentropy: 0.2160 - balanced_logloss: 0.3145
##### Fold 3
Use /tmp/tmp_n_b3op_ as temporary training directory
Reading training dataset...
Training dataset read in 0:00:01.977462. Found 555 examples.
Training model...
Model trained in 0:00:00.731456
Compiling model...


...

Model compiled.
1/1 [==============================] - 1s 656ms/step - loss: 0.0000e+00 - accuracy: 0.0000e+00 - binary_crossentropy: 0.2546 - balanced_logloss: 0.3383
##### Fold 9
Use /tmp/tmptmxd61vl as temporary training directory
Reading training dataset...
Training dataset read in 0:00:01.998719. Found 556 examples.
Training model...
Model trained in 0:00:00.824242
Compiling model...


[INFO 23-07-17 21:16:43.6092 UTC kernel.cc:1242] Loading model from path /tmp/tmptmxd61vl/model/ with prefix 53ab5a08252f48c7
[INFO 23-07-17 21:16:43.7430 UTC decision_forest.cc:660] Model loaded with 1000 root(s), 34142 node(s), and 113 input feature(s).
[INFO 23-07-17 21:16:43.7431 UTC abstract_model.cc:1311] Engine "RandomForestOptPred" built
[INFO 23-07-17 21:16:43.7431 UTC kernel.cc:1074] Use fast generic engine


Model compiled.
1/1 [==============================] - 1s 668ms/step - loss: 0.0000e+00 - accuracy: 0.0000e+00 - binary_crossentropy: 0.2580 - balanced_logloss: 0.3867
##### Fold 10
Use /tmp/tmpqhv3aa_k as temporary training directory
Reading training dataset...
Training dataset read in 0:00:01.973754. Found 556 examples.
Training model...
Model trained in 0:00:00.729179
Compiling model...


[INFO 23-07-17 21:16:50.2632 UTC kernel.cc:1242] Loading model from path /tmp/tmpqhv3aa_k/model/ with prefix ee2077d43faa4e4a
[INFO 23-07-17 21:16:50.4004 UTC decision_forest.cc:660] Model loaded with 1000 root(s), 34820 node(s), and 113 input feature(s).
[INFO 23-07-17 21:16:50.4005 UTC kernel.cc:1074] Use fast generic engine


Model compiled.
1/1 [==============================] - 1s 650ms/step - loss: 0.0000e+00 - accuracy: 0.0000e+00 - binary_crossentropy: 0.2148 - balanced_logloss: 0.2814
Type 1
fold_1: acc: 0.0000 loss: 0.2010 balanced loss: 0.2942
fold_2: acc: 0.0000 loss: 0.2160 balanced loss: 0.3145
fold_3: acc: 0.0000 loss: 0.2781 balanced loss: 0.4360
fold_4: acc: 0.0000 loss: 0.2768 balanced loss: 0.3325
fold_5: acc: 0.0000 loss: 0.2335 balanced loss: 0.3191
fold_6: acc: 0.0000 loss: 0.2744 balanced loss: 0.3483
fold_7: acc: 0.0000 loss: 0.2641 balanced loss: 0.3973
fold_8: acc: 0.0000 loss: 0.2546 balanced loss: 0.3383
fold_9: acc: 0.0000 loss: 0.2580 balanced loss: 0.3867
fold_10: acc: 0.0000 loss: 0.2148 balanced loss: 0.2814

Average accuracy: 0.0000  Average loss: 0.2471 Average balanced loss: 0.3448


# Submission

In [11]:
def create_submitions(submition: pd.DataFrame) -> pd.DataFrame:
    
    df = pd.DataFrame(data=np.zeros((len(submition.index),2)),index=submition.index, columns=['class_0','class_1'])
    
    df['class_1'] = submition_1.mean(axis=1)
    df['class_0'] = 1 - df['class_1']
    
    df.to_csv('/kaggle/working/submission.csv', index=True)
    
    return df

submition_1_final = create_submitions(submition=submition_1)

In [12]:
submition_1_final

,class_0,class_1
Id,,
00eed32682bb,0.656801,0.343199
010ebe33f668,0.656801,0.343199
02fa521e1838,0.656801,0.343199
040e15f562a2,0.656801,0.343199
046e85c7cc7f,0.656801,0.343199
